# Notebook for DeBERTa-v3-base

In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
from conllu import TokenList
import polars as pl
from stanza.models.common.doc import Token
from label_legends.preprocess import create_conllu, holdout, load_conllu, load_data, load_train, transform, load_vectorizer, reverse_vocabulary, vocabulary, ids_to_tokens, tokens_to_ids, vectorize_tokens, strip_stopwords
import logging

logging.basicConfig(level=logging.INFO)

# Importing dependencies for this specific model

In [55]:
import torch
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay
from time import time


In [56]:
from transformers import AutoModelForSequenceClassification, AutoConfig, DebertaV2Tokenizer, Trainer, TrainingArguments
MODEL_NAME = 'microsoft/deberta-v3-base'
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
config = AutoConfig.from_pretrained(MODEL_NAME)
tokenizer =DebertaV2Tokenizer.from_pretrained(MODEL_NAME)
#DO I need the tokenizer here?

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
load_data().collect()

id,rewire_id,text,annotator,label_sexist,label_category,label_vector,split,tokens
i64,str,str,i64,str,str,str,str,list[str]
0,"""sexism2022_english-0""",""" I wonder what keeps that witc…",17,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train""","[""i"", ""wonder"", … ""😄""]"
1,"""sexism2022_english-0""",""" I wonder what keeps that witc…",2,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train""","[""i"", ""wonder"", … ""😄""]"
10,"""sexism2022_english-100""","""Good for her! My grandson had …",3,"""not sexist""","""none""","""none""","""train""","[""good"", ""for"", … ""!""]"
100,"""sexism2022_english-10026""","""It is not insulting, it's **ex…",14,"""sexist""","""2. derogation""","""2.1 descriptive attacks""","""test""","[""it"", ""be"", … "".**""]"
1000,"""sexism2022_english-10297""","""The article said Madonna offer…",5,"""sexist""","""2. derogation""","""2.3 dehumanising attacks & ove…","""train""","[""the"", ""article"", … "".""]"
…,…,…,…,…,…,…,…,…
9995,"""sexism2022_english-12996""","""Shudder.. if you had to have s…",17,"""sexist""","""2. derogation""","""2.3 dehumanising attacks & ove…","""test""","[""shudder"", "".."", … "".""]"
9996,"""sexism2022_english-12997""","""You mean one that forces women…",6,"""not sexist""","""none""","""none""","""train""","[""you"", ""mean"", … ""?""]"
9997,"""sexism2022_english-12997""","""You mean one that forces women…",4,"""not sexist""","""none""","""none""","""train""","[""you"", ""mean"", … ""?""]"


In [58]:
val, tra = holdout()
tra = transform(tra)
val = transform(val)

# Convert 'label' column from string '0' and '1' to integer 0 and 1
tra = tra.with_columns(
    polars.col("label").cast(polars.Int32)
)
val = val.with_columns(
    polars.col("label").cast(polars.Int32)
)

train_texts = tra["text"].to_list()
train_labels = tra["label"].to_list()
val_texts = val["text"].to_list()
val_labels = val["label"].to_list()


In [59]:
train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=128,  # Adjust this as needed
    return_tensors="pt",
)
val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors="pt",
)


 33%|███▎      | 3675/11025 [15:43:17<31:26:34, 15.40s/it]



In [60]:
class SexistDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

    
    
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="micro")
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [61]:
train_dataset = SexistDataset(train_encodings, train_labels)
val_dataset = SexistDataset(val_encodings, val_labels)

In [62]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    evaluation_strategy="epoch",  # Keep this for evaluation at the end of each epoch
    logging_steps=100,  # Log less frequently than every step, but more often than every epoch
    save_strategy="epoch",  # Save checkpoints at the end of each epoch
    load_best_model_at_end=True,  # Load best model based on evaluation loss
    metric_for_best_model='eval_loss',  # Track evaluation loss for best model
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

#trainer.train()


/Users/retipeter/Documents/TU Wien/NLP/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/b6/1mflrv1s5s9b5pj5n4cg2hjw0000gn/T/ipykernel_88913/1078182425.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  1%|          | 119/11025 [12:16:45<1125:22:25, 371.48s/it]


In [63]:
#trainer.evaluate()


In [64]:

train_output = trainer.train()  # Returns a TrainOutput object
train_metrics = train_output.metrics  # Contains train_runtime, train_loss, etc.

# Assume trainer.evaluate() has been run and returned results
eval_results = trainer.evaluate()  # Contains eval_runtime, eval_accuracy, etc.

# Combine training and evaluation metrics
#all_metrics = {**train_metrics, **eval_results}
metrics = {**eval_results}

# Create a Polars DataFrame
df = pl.DataFrame(metrics)
df

  0%|          | 0/11025 [00:00<?, ?it/s]/var/folders/b6/1mflrv1s5s9b5pj5n4cg2hjw0000gn/T/ipykernel_88913/2940528382.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  1%|          | 100/11025 [00:36<1:06:37,  2.73it/s]

{'loss': 0.5642, 'grad_norm': 2.3384995460510254, 'learning_rate': 1.981859410430839e-05, 'epoch': 0.03}


  2%|▏         | 200/11025 [01:13<1:06:03,  2.73it/s]

{'loss': 0.5272, 'grad_norm': 15.648771286010742, 'learning_rate': 1.963718820861678e-05, 'epoch': 0.05}


  3%|▎         | 300/11025 [01:49<1:05:22,  2.73it/s]

...

{'eval_loss': 0.46143072843551636, 'eval_accuracy': 0.8668253968253968, 'eval_f1': 0.8668253968253968, 'eval_precision': 0.8668253968253968, 'eval_recall': 0.8668253968253968, 'eval_runtime': 145.0987, 'eval_samples_per_second': 86.837, 'eval_steps_per_second': 10.855, 'epoch': 3.0}


100%|██████████| 11025/11025 [1:24:15<00:00,  2.18it/s]
/var/folders/b6/1mflrv1s5s9b5pj5n4cg2hjw0000gn/T/ipykernel_88913/2940528382.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'train_runtime': 5055.2793, 'train_samples_per_second': 17.447, 'train_steps_per_second': 2.181, 'train_loss': 0.3528726281060113, 'epoch': 3.0}


100%|██████████| 1575/1575 [02:25<00:00, 10.81it/s]


eval_loss,eval_accuracy,eval_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
f64,f64,f64,f64,f64,f64,f64,f64,f64
0.347226,0.860238,0.860238,0.860238,0.860238,145.968,86.32,10.79,3.0
